# WORD GRAPH PROJECT - step 1

We will use the work done here 

https://www.kaggle.com/datasets/moxxis/harry-potter-lstm?resource=download&select=Harry_Potter_all_books_preprocessed.txt

so thank to this boy to provide all the data we searched and to prepocess even them (this should help a lot). 

We start by reading everything.

In [4]:
HP_file = open("HP_books.txt", "r")
HP_text = HP_file.read()

Now we split everything and count how many words there are

In [14]:
HP_words = HP_text.split(" ")
len(HP_words)

1087554

And we count the numbers of single words.

In [15]:
HP_set_words = set(HP_words)
len(HP_set_words)

31967

This is something really cool to notice. We now there are approximetely 170.000 words in the english vocabulary and this means that HP really minimize the space of the words (relatively talking).
Now we want to count the occurrencies of the single words. 

In [24]:
HP_occourrencies_dict = {}
for word in HP_words:
    if word in HP_occourrencies_dict.keys():
        HP_occourrencies_dict[word] += 1
    else:
        HP_occourrencies_dict[word] = 1
HP_sorted_occourrencies_dict = dict(sorted(HP_occourrencies_dict.items(), key=lambda item: item[1], reverse=False))

In [23]:
k = 0
lim = 10
for unique_word in HP_sorted_occourrencies_dict:
    if k < lim:
        print(f"{unique_word} occourred {HP_sorted_occourrencies_dict[unique_word]}")
        k = k + 1

NameError: name 'HP_sorted_occourrencies_dict' is not defined